In [6]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [18]:
df = pd.read_parquet("C:\\Users\\Tobi\\Desktop\\New folder\\git\\Energy_Load_Profile_Classification\\Energy_Load_Profile_Classification\\main.parquet")

In [26]:
df = df[['Time','system_kW']]

In [27]:
df['Time'] = pd.to_datetime(df['Time'])

<ipython-input-27-1bfd8981fe14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = pd.to_datetime(df['Time'])


In [28]:
df['new_date'] = [d.date() for d in df['Time']]
df['new_time'] = [d.time() for d in df['Time']]

<ipython-input-28-dd7829446a33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_date'] = [d.date() for d in df['Time']]
<ipython-input-28-dd7829446a33>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_time'] = [d.time() for d in df['Time']]


In [38]:
import pandas as pd

# Get week number, with pd.DateOffset used to start the week on Sunday
week_date_values = df["Time"].dt.isocalendar()
df["week_date"] = (df['Time'] - pd.to_timedelta(df['Time'].dt.dayofweek, unit='d') + pd.DateOffset(days=-1)).dt.strftime("%Y-%m-%d")

# Combine with "(week n)" values
df["week_date"] = df["week_date"] + " " + "(week " + week_date_values["week"].astype(str) + ")"
df

<ipython-input-38-6d77f126da1b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["week_date"] = (df['Time'] - pd.to_timedelta(df['Time'].dt.dayofweek, unit='d') + pd.DateOffset(days=-1)).dt.strftime("%Y-%m-%d")
<ipython-input-38-6d77f126da1b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["week_date"] = df["week_date"] + " " + "(week " + week_date_values["week"].astype(str) + ")"


,Time,system_kW,new_date,new_time,week_date
0,2022-09-17 12:38:00,40.068,2022-09-17,12:38:00,2022-09-11 (week 37)
1,2022-09-17 12:39:00,35.686,2022-09-17,12:39:00,2022-09-11 (week 37)
2,2022-09-17 12:40:00,36.587,2022-09-17,12:40:00,2022-09-11 (week 37)
3,2022-09-17 12:48:00,32.229,2022-09-17,12:48:00,2022-09-11 (week 37)
4,2022-09-17 12:49:00,29.376,2022-09-17,12:49:00,2022-09-11 (week 37)
...,...,...,...,...,...
194020,2023-01-31 23:55:00,58.405,2023-01-31,23:55:00,2023-01-29 (week 5)
194021,2023-01-31 23:56:00,57.992,2023-01-31,23:56:00,2023-01-29 (week 5)
194022,2023-01-31 23:57:00,56.412,2023-01-31,23:57:00,2023-01-29 (week 5)
194023,2023-01-31 23:58:00,58.341,2023-01-31,23:58:00,2023-01-29 (week 5)


In [59]:
# data re-sampled based on an hour
df_by_hr = df.resample('H', on='Time').system_kW.max()

In [60]:
df_by_hr = pd.DataFrame(df_by_hr).reset_index()

In [61]:
df_by_hr

,Time,system_kW
0,2022-09-17 12:00:00,46.666
1,2022-09-17 13:00:00,51.045
2,2022-09-17 14:00:00,45.432
3,2022-09-17 15:00:00,47.489
4,2022-09-17 16:00:00,59.146
...,...,...
3271,2023-01-31 19:00:00,87.146
3272,2023-01-31 20:00:00,82.882
3273,2023-01-31 21:00:00,76.380
3274,2023-01-31 22:00:00,62.340


In [62]:
import pandas as pd

# Get week number, with pd.DateOffset used to start the week on Sunday
week_date_values = df_by_hr["Time"].dt.isocalendar()
df_by_hr["week_date"] = (df_by_hr['Time'] - pd.to_timedelta(df_by_hr['Time'].dt.dayofweek, unit='d') + pd.DateOffset(days=-1)).dt.strftime("%Y-%m-%d")

# Combine with "(week n)" values
df_by_hr["week_date"] = df_by_hr["week_date"] + " " + "(week " + week_date_values["week"].astype(str) + ")"
df_by_hr

,Time,system_kW,week_date
0,2022-09-17 12:00:00,46.666,2022-09-11 (week 37)
1,2022-09-17 13:00:00,51.045,2022-09-11 (week 37)
2,2022-09-17 14:00:00,45.432,2022-09-11 (week 37)
3,2022-09-17 15:00:00,47.489,2022-09-11 (week 37)
4,2022-09-17 16:00:00,59.146,2022-09-11 (week 37)
...,...,...,...
3271,2023-01-31 19:00:00,87.146,2023-01-29 (week 5)
3272,2023-01-31 20:00:00,82.882,2023-01-29 (week 5)
3273,2023-01-31 21:00:00,76.380,2023-01-29 (week 5)
3274,2023-01-31 22:00:00,62.340,2023-01-29 (week 5)
